In [1]:
# This is attention based residual network architecture design
from __future__ import print_function
import keras
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import AveragePooling2D,GlobalAveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.layers import AveragePooling2D, Input, Flatten, Add
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import glorot_uniform
from datetime import datetime

# attention modules importing
import attention_modules
from attention_modules import attach_attention_module


Using TensorFlow backend.


Instructions for updating:
Use `tf.config.run_functions_eagerly` instead of the experimental version.


In [2]:
def identity_block(X, f, filters, stage, block, attention_module=None):
    """
    Implementation of the identity block for a RseNet model
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # attention module
    #if attention_module is not None:
    #    Y = attach_attention_module(X, attention_module)
    #X = Add()([X_shortcut, Y])
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2, attention_module=None):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # attention modules added
    if attention_module is not None:
        X = attach_attention_module(X, attention_module)
    X = Add()([X, X_shortcut])
    
    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    #X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [4]:
def LW_ResNet20(input_shape=(224, 224, 3), classes=11, attention_module=None):
    """
    Implementation of the custom ResNet20 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> CONVBLOCK -> IDBLOCK*1
    -> CONVBLOCK -> IDBLOCK*1 -> CONVBLOCK -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(16, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[16, 16, 64], stage=2, block='a', s=1)
   
    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [32, 32, 128], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [32, 32, 128], stage=3, block='b')
       
    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 4, block='a', s = 2, attention_module=attention_module)
    X = identity_block(X, 3, [64, 64, 256], stage=4, block='b')
   
    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 5, block='a', s = 2)
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    #X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    #X = AveragePooling2D((2,2), name="avg_pool")(X)
    X = GlobalAveragePooling2D(name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    #X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc_' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    # for naming the model
    if attention_module is not None:
        attention_name = '_'+attention_module.split('_')[0]
    else:
        attention_name = ''
    # Create model
    model = Model(inputs = X_input, outputs = X, name='LW_ResNet20'+attention_name)

    return model

In [7]:
model = LW_ResNet20(input_shape = (256, 256, 3), classes = 11, attention_module='da_module')
model.summary()


Model: "LW_ResNet20_da"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
zero_padding2d_2 (ZeroPadding2D (None, 262, 262, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 128, 128, 16) 2368        zero_padding2d_2[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 128, 128, 16) 64          conv1[0][0]                      
_____________________________________________________________________________________

In [8]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


In [9]:
# image preprocessing
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   rotation_range=40,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)


batch_size = 32

train_data_dir = "D:/Anil/Dataset/tomato/train"     # directory of training data

test_data_dir = "D:/Anil/Dataset/tomato/val"      # directory of Validation data

training_set = train_datagen.flow_from_directory(train_data_dir,
                                                 target_size=(256, 256),
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

test_set = test_datagen.flow_from_directory(test_data_dir,
                                            target_size=(256, 256),
                                            batch_size=batch_size,
                                            class_mode='categorical')

print(training_set.class_indices)



Found 15156 images belonging to 11 classes.
Found 1891 images belonging to 11 classes.
{'bacterial_spot': 0, 'early_blight': 1, 'fusarium_wilt': 2, 'healthy': 3, 'late_blight': 4, 'leaf_mold': 5, 'mosaic_virus': 6, 'septoria_leaf_spot': 7, 'spider_mites': 8, 'target_spot': 9, 'yellow_leaf_curl_virus': 10}


In [10]:
# checkpoint
weightpath = "CBAM-keras/saved_models/LW_ResNet20_da_module_tomato_adam_100_epochs.hdf5"
checkpointer = ModelCheckpoint(weightpath, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

epochs = 100

start_time = datetime.now()
print("start time: "+str(start_time))

#fitting images to CNN
history = model.fit(training_set,
                         steps_per_epoch=training_set.samples//batch_size,
                         validation_data=test_set,
                         epochs=epochs,
                         validation_steps=test_set.samples//batch_size,
                         callbacks=[checkpointer])#callbacks_list)


# #fitting images to CNN
# history = classifier.fit_generator(training_set,
#                                    steps_per_epoch=training_set.samples//batch_size,
#                                    validation_data=test_set,
#                                    epochs=10,
#                                    validation_steps=test_set.samples//batch_size)

# Stop time
stop_time = datetime.now()
print("stop time: "+str(stop_time))

total_time = stop_time - start_time
print("Total time: "+str(total_time))


start time: 2021-12-29 17:15:38.465797


C:\Users\USER\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\data\ops\dataset_ops.py:3504: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/100
473/473 [==============================] - 146s 304ms/step - loss: 1.0511 - accuracy: 0.6653 - val_loss: 1.7356 - val_accuracy: 0.6186

Epoch 00001: val_loss improved from inf to 1.73563, saving model to CBAM-keras/saved_models\LW_ResNet20_da_module_tomato_adam_100_epochs.hdf5
Epoch 2/100
473/473 [==============================] - 136s 288ms/step - loss: 0.5213 - accuracy: 0.8234 - val_loss: 1.3700 - val_accuracy: 0.6335

Epoch 00002: val_loss improved from 1.73563 to 1.36998, saving model to CBAM-keras/saved_models\LW_ResNet20_da_module_tomato_adam_100_epochs.hdf5
Epoch 3/100
473/473 [==============================] - 136s 288ms/step - loss: 0.3907 - accuracy: 0.8678 - val_loss: 7.6756 - val_accuracy: 0.3120

Epoch 00003: val_loss did not improve from 1.36998
Epoch 4/100
473/473 [==============================] - 136s 287ms/step - loss: 0.3675 - accuracy: 0.8732 - val_loss: 1.0826 - val_accuracy: 0.7066

Epoch 00004: val_loss improved from 1.36998 to 1.08258, saving model 

473/473 [==============================] - 135s 286ms/step - loss: 0.0583 - accuracy: 0.9817 - val_loss: 0.1670 - val_accuracy: 0.9449

Epoch 00038: val_loss did not improve from 0.07721
Epoch 39/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0567 - accuracy: 0.9799 - val_loss: 1.5360 - val_accuracy: 0.8130

Epoch 00039: val_loss did not improve from 0.07721
Epoch 40/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0651 - accuracy: 0.9782 - val_loss: 0.3771 - val_accuracy: 0.9168

Epoch 00040: val_loss did not improve from 0.07721
Epoch 41/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0552 - accuracy: 0.9820 - val_loss: 13.7655 - val_accuracy: 0.2977

Epoch 00041: val_loss did not improve from 0.07721
Epoch 42/100
473/473 [==============================] - 135s 286ms/step - loss: 0.0568 - accuracy: 0.9781 - val_loss: 0.1204 - val_accuracy: 0.9629

Epoch 00042: val_loss did not improve from 0.07721
Epoch 43/100


Epoch 00078: val_loss improved from 0.03889 to 0.02869, saving model to CBAM-keras/saved_models\LW_ResNet20_da_module_tomato_adam_100_epochs.hdf5
Epoch 79/100
473/473 [==============================] - 137s 289ms/step - loss: 0.0283 - accuracy: 0.9909 - val_loss: 0.2763 - val_accuracy: 0.9248

Epoch 00079: val_loss did not improve from 0.02869
Epoch 80/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0286 - accuracy: 0.9897 - val_loss: 9.3146 - val_accuracy: 0.5201

Epoch 00080: val_loss did not improve from 0.02869
Epoch 81/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0366 - accuracy: 0.9860 - val_loss: 0.1861 - val_accuracy: 0.9529

Epoch 00081: val_loss did not improve from 0.02869
Epoch 82/100
473/473 [==============================] - 136s 287ms/step - loss: 0.0221 - accuracy: 0.9907 - val_loss: 4.1814 - val_accuracy: 0.7119

Epoch 00082: val_loss did not improve from 0.02869
Epoch 83/100
473/473 [==============================]

In [11]:
# another way to save it.
import pandas as pd
hist_df = pd.DataFrame(history.history) 
hist_csv_file = "CBAM-keras/saved_models/LW_ResNet20_da_module_tomato_adam_100_epochs.csv"
# hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w', newline='') as f:
    hist_df.to_csv(f)
